In [1]:
import pandas as pd
import os

# 1. Ler Arquivos

In [3]:
# Sobe um nível no diretório
os.chdir("..")

In [4]:
# Armazena o novo diretório de trabalho atual
diretorio_atual = os.getcwd()
print(diretorio_atual)

C:\Workspace\Streamlit\api_contabil_TESTE


In [5]:
# Caminho dos diretorios
dir_preprocessed_data = os.path.join(diretorio_atual, "data_cvm\preprocessed_II")

In [6]:
# Define o diretório
os.chdir(dir_preprocessed_data)

In [7]:
# Ler dados trimestral
df_trim = pd.read_csv("equatorial_resultado_trim.csv")

In [8]:
# Ler dados anual
df_anual = pd.read_csv("equatorial_resultado_anual.csv")

In [9]:
df = pd.concat([df_anual, df_trim], ignore_index=True)

Os dados trimestrais e anuais foram concatenados para incluir o 4 trimestre, que será calculado posteriormente. \ 
O processamento deve ser realizado em ambos os dados, anuais e trimestrais(concatenado).

# 2. Análise Exploratória

## 2.1 Verficando se há meses duplicados 

In [10]:
teste = df[(df['CD_CONTA'] == '3.01') & (df['ANO'] == 2022)] 

In [11]:
display(teste)

,CNPJ_CIA,DENOM_CIA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,DT_INI_EXERC,TIPO_DEM,ANO,MES
395,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-12-31,3.01,Receita de Venda de Bens e/ou Serviços,27132507.0,2022-01-01,Demonstração do Resultado,2022,12
2554,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-03-31,3.01,Receita de Venda de Bens e/ou Serviços,5843189.0,2022-01-01,Demonstração do Resultado,2022,3
2593,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-06-30,3.01,Receita de Venda de Bens e/ou Serviços,12335295.0,2022-01-01,Demonstração do Resultado,2022,6
2594,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-06-30,3.01,Receita de Venda de Bens e/ou Serviços,6492106.0,2022-04-01,Demonstração do Resultado,2022,6
2671,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-09-30,3.01,Receita de Venda de Bens e/ou Serviços,19215572.0,2022-01-01,Demonstração do Resultado,2022,9
2672,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-09-30,3.01,Receita de Venda de Bens e/ou Serviços,6880277.0,2022-07-01,Demonstração do Resultado,2022,9


Os meses 6 e 9 possuem valores onde o exercício começa em janeiro do ano vigente e valores onde começam no fim do trimestre anterior.

# 3. Manipulação dos Dados

## 3.1 Filtrando as Contas Por Nível

In [12]:
# Filtra contas dos níveis mais altos
df_anual = df_anual[df_anual['CD_CONTA'].str.len() <= 7]
df = df[df['CD_CONTA'].str.len() <= 7]

## 3.1 Deletando os meses 6 e 9 com inicio do exercicio em janeiro

### 3.1.1 Criando coluna ini_mes

In [13]:
# Converte coluna DT_REFER para o tipo datetime
df['DT_INI_EXERC'] = pd.to_datetime(df['DT_INI_EXERC'])

# Separa o mês e cria a coluna mês
df['INI_MES'] = pd.DatetimeIndex(df['DT_INI_EXERC']).month

In [14]:
# Crie uma condição para identificar as linhas que deseja excluir
condicao = (df["MES"].isin([6, 9])) & (df["INI_MES"] == 1)
# print(condicao)

df_droped = df.drop(df[condicao].index)
# display(df_drop.tail())

# VALIDAÇÃO DA EXCLUSÃO - resultado vazio esta ok
print(df_droped[(df_droped['MES'] == 9) & (df_droped["INI_MES"]== 1)])

Empty DataFrame
Columns: [CNPJ_CIA, DENOM_CIA, DT_FIM_EXERC, CD_CONTA, DS_CONTA, VL_CONTA, DT_INI_EXERC, TIPO_DEM, ANO, MES, INI_MES]
Index: []


In [15]:
teste = df_droped[(df_droped['CD_CONTA'] == '3.01') & (df_droped['ANO'] == 2022)] 
display(teste)

,CNPJ_CIA,DENOM_CIA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,DT_INI_EXERC,TIPO_DEM,ANO,MES,INI_MES
395,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-12-31,3.01,Receita de Venda de Bens e/ou Serviços,27132507.0,2022-01-01,Demonstração do Resultado,2022,12,1
2554,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-03-31,3.01,Receita de Venda de Bens e/ou Serviços,5843189.0,2022-01-01,Demonstração do Resultado,2022,3,1
2594,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-06-30,3.01,Receita de Venda de Bens e/ou Serviços,6492106.0,2022-04-01,Demonstração do Resultado,2022,6,4
2672,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,2022-09-30,3.01,Receita de Venda de Bens e/ou Serviços,6880277.0,2022-07-01,Demonstração do Resultado,2022,9,7


## 3.2 Deletando as Colunas

In [18]:
df_droped_column_anual = df_anual.drop(
    columns=['DT_FIM_EXERC','DT_INI_EXERC'])

In [19]:
df_droped_column = df_droped.drop(
    columns=['DT_FIM_EXERC', 'INI_MES','DT_INI_EXERC'])

## 3.3 Inserindo a Coluna Período

### 3.3.1 Anual

In [20]:
df_droped_column_anual['PERIODO'] = 'ANUAL'

### 3.3.2 Trimestral

In [22]:
# Função para inserir a coluns Período
def insere_periodo_trimestral(df):
    df = df.copy()

    # Dicionário para mapear os meses aos trimestres
    mes_para_trimestre = {3: 1, 6: 2, 9: 3, 12: 4}
    
    # Filtrar apenas os anos e meses relevantes
    anos_relevantes = list(range(2011, 2025))
    meses_relevantes = [3, 6, 9, 12]

    # Aplicar a lógica de período trimestral
    df = df[(df['ANO'].isin(anos_relevantes)) & (df['MES'].isin(meses_relevantes))]
    df['PERIODO'] = df.apply(lambda row: f"{mes_para_trimestre[row['MES']]}T{row['ANO']}", axis=1)
    
    return df

In [23]:
# Inserindo a coluna período
df_droped_column = insere_periodo_trimestral(df_droped_column)

In [25]:
df_droped_column[(df_droped_column["ANO"] == 2023) & (df_droped_column['CD_CONTA'] == "3.02")]

,CNPJ_CIA,DENOM_CIA,CD_CONTA,DS_CONTA,VL_CONTA,TIPO_DEM,ANO,MES,PERIODO
437,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-28842545.0,Demonstração do Resultado,2023,12,4T2023
2750,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-7380981.0,Demonstração do Resultado,2023,3,1T2023
2793,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-6438133.0,Demonstração do Resultado,2023,6,2T2023
2871,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-7055107.0,Demonstração do Resultado,2023,9,3T2023


## 3.4 Calculando 4º Trimestre

In [26]:
# Cria uma cópia do dataframe
df_calculado = df_droped_column.copy()

In [27]:
# Obter os anos e contas únicos
anos = df_calculado['ANO'].unique()
contas = df_calculado['CD_CONTA'].unique()

In [28]:
# Soma os valores do mes 3, 6 e 9 e subtrai o valor do mês 12 
for ano in anos:
    for conta in contas:
        # Filtra o DataFrame com base nas condições de conta e ano
        filtro = (df_calculado['CD_CONTA'] == conta) & (df_calculado['ANO'] == ano)
        
        # Filtra as linhas onde o MES é igual a 3, 6 ou 9
        filtro_mes_369 = filtro & df_calculado['MES'].isin([3, 6, 9])
        
        # Obtém a soma dos valores da coluna VL_CONTA onde MES é igual a 3, 6 ou 9
        soma_mes_369 = df_calculado[filtro_mes_369]['VL_CONTA'].sum()
        
        # Filtra as linhas onde o MES é igual a 12
        filtro_mes_12 = filtro & (df_calculado['MES'] == 12)
        
        # Subtrai a soma dos meses 3, 6 e 9 do valor do mês 12
        df_calculado.loc[filtro_mes_12, 'VL_CONTA'] = df_calculado.loc[filtro_mes_12, 'VL_CONTA'] - soma_mes_369

print("DataFrame atualizado:")
# display(df_drop_column.head())

DataFrame atualizado:


In [29]:
df_calculado[(df_calculado["ANO"] == 2022) & (df_calculado['CD_CONTA'] == "3.02")]

,CNPJ_CIA,DENOM_CIA,CD_CONTA,DS_CONTA,VL_CONTA,TIPO_DEM,ANO,MES,PERIODO
398,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-5458101.0,Demonstração do Resultado,2022,12,4T2022
2557,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-3844293.0,Demonstração do Resultado,2022,3,1T2022
2600,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-4380488.0,Demonstração do Resultado,2022,6,2T2022
2678,03.220.438/0001-73,EQUATORIAL ENERGIA S.A.,3.02,Custo dos Bens e/ou Serviços Vendidos,-4882854.0,Demonstração do Resultado,2022,9,3T2022


Resultado esperado: \
1T2022 = -3844293.0 \
2T2022 = -4380488.0 \
3T2022 = -4882854.0 \
4T2022 = -5458101.0	

## 3.5 Concatenando Dados Anuais e Trimestrais

In [30]:
df_final = pd.concat([df_calculado, df_droped_column_anual], ignore_index=True)

## 3.6 Alterando o Nome das Colunas

In [31]:
colunas = ['CNPJ','EMPRESA','CONTA','DESCRIÇÃO','VALOR','DEMONSTRATIVO','ANO', 'MES', 'PERIODO']
df_final.columns = colunas

# 4. Exportando os Dados

In [32]:
# Define o diretório de dados processados
dir_processed_data = os.path.join(diretorio_atual, "data_cvm\processed")
os.chdir(dir_processed_data)

In [33]:
df_final.to_csv('demonstracao_resultado_completo.csv', index=False)